# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [ ]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
#
import pandas as pd
for crunchbase in  pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize = 5000):
  print(crunchbase.head(5))



In [ ]:
crunchbase.memory_usage(deep = True)


The total memory footprint of all of the chunks combined.

In [4]:

memory_footprints = []
crunchbase_iter = pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize = 5000)
for chunk in crunchbase_iter:
    memory_footprints.append(crunchbase.memory_usage(deep=True).sum()/(1024*1024))
print(sum(memory_footprints))
 

29.30085849761963


Which column(s) we can drop because they aren't useful for analysis.

In [ ]:
crunchbase = pd.read_csv("crunchbase-investments.csv", encoding='latin1')
crunchbase.drop(['investor_country_code', 'investor_state_code','investor_category_code', 'company_city','raised_amount_usd','investor_city','company_state_code','company_category_code',], axis = 1)

Each column's missing value counts.

In [ ]:
print( crunchbase.isnull().sum())

## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [45]:
# Your code goes here

Identify the types for each column

In [ ]:
print(crunchbase.info())

Identify the numeric columns we can represent using more space efficient types. For text columns:

In [23]:
numeric= ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_cols = crunchbase.select_dtypes(include=numeric).columns
numeric_cols
# Write you code below


Index(['funded_year', 'raised_amount_usd'], dtype='object')

Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.

In [10]:
crunchbase_iter = pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize = 5000)
for crunchbase in crunchbase_iter:
    chunk_vc = crunchbase.value_counts()
    print(chunk_vc)


company_permalink                           company_name                       company_category_code  company_country_code  company_state_code  company_region  company_city   investor_permalink                 investor_name             investor_category_code  investor_country_code  investor_state_code  investor_region  investor_city  funding_round_type  funded_at   funded_month  funded_quarter  funded_year  raised_amount_usd
/company/massachusetts-clean-energy-center  Massachusetts Clean Energy Center  cleantech              USA                   MA                  Boston          Boston         /company/north-shore-innoventures  North Shore InnoVentures  biotech                 USA                    MA                   Boston           Beverly        angel               2013-09-05  2013-09       2013-Q3         2013         300000.0             2
/company/1000memories                       1000memories                       web                    USA                   CA           

See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.

## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [18]:
# Your code goes here
import sqlite3
import pandas as pd
crunchbase_iter = pd.read_csv("crunchbase-investments.csv", encoding='latin1', chunksize = 5000)
conn = sqlite3.connect('crunchbase_iter')
for chunk in crunchbase_iter:
    chunk.to_sql("exhibitions", conn, if_exists='append', index=False)
    results_df = pd.read_sql('PRAGMA table_info(exhibitions);', conn)
print(results_df)

    cid                    name     type  notnull dflt_value  pk
0     0       company_permalink     TEXT        0       None   0
1     1            company_name     TEXT        0       None   0
2     2   company_category_code     TEXT        0       None   0
3     3    company_country_code     TEXT        0       None   0
4     4      company_state_code     TEXT        0       None   0
5     5          company_region     TEXT        0       None   0
6     6            company_city     TEXT        0       None   0
7     7      investor_permalink     TEXT        0       None   0
8     8           investor_name     TEXT        0       None   0
9     9  investor_category_code     TEXT        0       None   0
10   10   investor_country_code     TEXT        0       None   0
11   11     investor_state_code     TEXT        0       None   0
12   12         investor_region     TEXT        0       None   0
13   13           investor_city     TEXT        0       None   0
14   14      funding_roun

## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.

In [27]:
# Your code goes here
q = 'select raised_amount_usd, count(*) as counts from exhibitions group by raised_amount_usd  order by counts desc;'
eid_counts = pd.read_sql(q, conn)
print(eid_counts[:10])

   raised_amount_usd  counts
0                NaN   10144
1         10000000.0    6044
2          1000000.0    5138
3          5000000.0    4674
4         15000000.0    4158
5         20000000.0    3827
6          2000000.0    3670
7          1500000.0    3647
8          3000000.0    3543
9          6000000.0    3361


In [19]:
q = 'select raised_amount_usd , count(*) as counts from exhibitions group by raised_amount_usd  order by counts asc;'
eid_counts = pd.read_sql(q, conn)
print(eid_counts[:10])

   raised_amount_usd  counts
0             6000.0       2
1            11600.0       2
2            11800.0       2
3            15282.0       2
4            16500.0       2
5            22500.0       2
6            28000.0       2
7            30653.0       2
8            50206.0       2
9            53000.0       2


Which category of company attracted the most investments?

In [31]:
q = 'select company_category_code , count(*) as counts from exhibitions group by raised_amount_usd  order by counts desc;'
eid_counts = pd.read_sql(q, conn)
print(eid_counts[:10])


  company_category_code  counts
0             ecommerce   10144
1                mobile    6044
2             education    5138
3           advertising    4674
4              software    4158
5             analytics    3827
6                 other    3670
7             nonprofit    3647
8                mobile    3543
9                design    3361


Which investor contributed the most money (across all startups)?

In [33]:
q = 'select investor_name , count(*) as counts from exhibitions group by raised_amount_usd and company_name order by counts desc;'
eid_counts = pd.read_sql(q, conn)
print(eid_counts[:10])

      investor_name  counts
0      Zorik Gordon  152613
1        Zach Klein     935
2  Michael Broukhim      62


Which investors contributed the most money per startup?

In [44]:
q = 'select investor_name,raised_amount_usd , count(*) as counts from exhibitions group by raised_amount_usd order by raised_amount_usd  desc;'
eid_counts = pd.read_sql(q, conn)
print(eid_counts[:10])

                   investor_name  raised_amount_usd  counts
0                    Time Warner       3.200000e+09      10
1         Marlin Equity Partners       2.600000e+09       3
2                  Goldman Sachs       1.500000e+09      19
3      Oak Hill Capital Partners       1.050000e+09       6
4                         Google       1.000000e+09       2
5  Technology Crossover Ventures       9.500000e+08      30
6           Eagle River Holdings       9.200000e+08      11
7           Siemens PLM Software       7.500000e+08       2
8                  Goldman Sachs       5.650000e+08       3
9      U.S. Department of Energy       5.350000e+08       3


Which funding round was the most popular? Which was the least popular?

In [55]:
q = 'select funding_round_type , count(*) as counts from exhibitions group by funding_round_type order by counts desc;'
eid_counts = pd.read_sql(q, conn)
print(eid_counts[:10])


  funding_round_type  counts
0           series-a   40851
1          series-c+   31706
2            venture   25947
3           series-b   25680
4              angel   25533
5              other    2755
6     private-equity    1033
7           post-ipo      82
8       crowdfunding      14
9               None       9


In [56]:
q = 'select funding_round_type , count(*) as counts from exhibitions group by funding_round_type order by counts asc;'
eid_counts = pd.read_sql(q, conn)
print(eid_counts[:10])

  funding_round_type  counts
0               None       9
1       crowdfunding      14
2           post-ipo      82
3     private-equity    1033
4              other    2755
5              angel   25533
6           series-b   25680
7            venture   25947
8          series-c+   31706
9           series-a   40851
